In [1]:
import pandas as pd
from PIL import Image
import codecs
import os, re

import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg') # 输出为svg

df = pd.read_csv('测试 第 1 轮/test1-utf8.csv')
df['filename'] = 'input/训练集/image/' + df['filename'] # 改为本地路径

# 数据样例

In [2]:
df[df['filename'] == df['filename'].iloc[629]]

,index,question_id,filename,question_text
628,629,Q00629,input/训练集/image/eb12d5c8c1cf5a82a308e3244ed22d...,这是什么报？
629,630,Q00630,input/训练集/image/eb12d5c8c1cf5a82a308e3244ed22d...,中国外交部在3月18日凌晨宣布了什么？
630,631,Q00631,input/训练集/image/eb12d5c8c1cf5a82a308e3244ed22d...,2018年12月美方要求了哪些？
631,632,Q00632,input/训练集/image/eb12d5c8c1cf5a82a308e3244ed22d...,2020年2月将5家中国媒体驻美机构被列为什么？
632,633,Q00633,input/训练集/image/eb12d5c8c1cf5a82a308e3244ed22d...,5家中国媒体在美国的中国籍雇佣员将有什么变化？
633,634,Q00634,input/训练集/image/eb12d5c8c1cf5a82a308e3244ed22d...,右上角是数字几？
634,635,Q00635,input/训练集/image/eb12d5c8c1cf5a82a308e3244ed22d...,右上角什么汉字？
635,636,Q00636,input/训练集/image/eb12d5c8c1cf5a82a308e3244ed22d...,左下角什么内容？
636,637,Q00637,input/训练集/image/eb12d5c8c1cf5a82a308e3244ed22d...,外交部宣布对什么采取措施？
637,638,Q00638,input/训练集/image/eb12d5c8c1cf5a82a308e3244ed22d...,针对美方对中国记者在哪些方面采取其实性别措施？


In [3]:
df

,index,question_id,filename,question_text
0,1,Q00001,input/训练集/image/2-603986_20190430_4_119.jpg,证书有效期至什么时候？
1,2,Q00002,input/训练集/image/2-603986_20190430_4_119.jpg,此单据经什么部门审查，批准？
2,3,Q00003,input/训练集/image/2-603986_20190430_4_119.jpg,图片右上角的阿拉伯数字是什么？
3,4,Q00004,input/训练集/image/2-603986_20190430_4_119.jpg,图片正上方的文字是什么/
4,5,Q00005,input/训练集/image/2-603986_20190430_4_119.jpg,图片左下角的文字是什么内容？
...,...,...,...,...
8610,8611,Q08611,input/训练集/image/b1867b04e902f91846a416fdaee3de...,该报纸正上方中间的一行文字的内容是什么？
8611,8612,Q08612,input/训练集/image/b1867b04e902f91846a416fdaee3de...,该报纸中的周晨指的是什么？
8612,8613,Q08613,input/训练集/image/b1867b04e902f91846a416fdaee3de...,该报纸中什么强硬拒绝了土耳其要求？
8613,8614,Q08614,input/训练集/image/b1867b04e902f91846a416fdaee3de...,什么问题谈不拢？


In [ ]:
# 对于所有的数据集，迭代每一行
# 步骤1：判断OCR是否识别

from tqdm import tqdm

test_result = []
for row in tqdm(df.iloc[:].iterrows()):
    qs = row[1].question_text
    # ocr是否识别成功
    if not os.path.exists('ocr_result/'+os.path.basename(row[1]['filename'])[:-4] + '.txt'):
        continue
    
    # 读取ocr识别结果
    ocrs = codecs.open('ocr_result/'+os.path.basename(row[1]['filename'])[:-4] + '.txt').readlines()[:]
    # 文字
    ocr_text = [x.split('\t')[1].strip() for x in ocrs]
    # 文本框
    ocr_box = [x.split('\t')[0].strip() for x in ocrs]
    
    candicate_text = None
    if re.findall('什么药品', qs):
#         pass
        candicate_text = ocr_text[0]
        
    elif re.findall('说明书来源于哪里', qs):
        candicate_text = list(set([x for x in ocr_text if '说明书' in x]))
        candicate_text = [x for x in candicate_text if '说明书' in x][0]
        candicate_text = candicate_text.replace('说明书', '').replace('来源', '').replace('：', '')
    elif re.findall('什么大学什么专业', qs):
        candicate_text = list(set([x for x in ocr_text if '大学' in x and '专业' in x]))
    elif re.findall('什么大学', qs):
        candicate_text = list(set([x for x in ocr_text if re.findall('大学', x)]))
        if len(candicate_text) == 0:
            continue
        
    elif re.findall('什么专业', qs):
        candicate_text = list(set([x for x in ocr_text if re.findall('本科', x)]))
        
    elif re.findall('邮箱', qs):
        candicate_text = list(set([x for x in ocr_text if re.findall('@', x)]))


    if candicate_text is None:
        test_result.append({'questionId': row[1].question_id, 'answer': ''})
    elif isinstance(candicate_text, list):
        if len(candicate_text) > 0:
            test_result.append({'questionId': row[1].question_id, 'answer': candicate_text[0]})
        else:
            test_result.append({'questionId': row[1].question_id, 'answer': ''})
    else:
        test_result.append({'questionId': row[1].question_id, 'answer': candicate_text})
    

        # break
    # break

In [ ]:
import json
with open('answer.json', 'w') as up:
    json.dump(test_result, up)

In [ ]:
!zip -r answer.zip answer.json

In [ ]:
df

# Bert

In [11]:
train_df = pd.read_csv('input/train-utf8.csv')
train_df['filename'] = 'input/训练集/image/' + train_df['filename'] # 改为本地路径

In [12]:
def read_ocr(row):
    if not os.path.exists('ocr_result/'+os.path.basename(str(row[1]['filename']))[:-4] + '.txt'):
        return ''
    
    ocrs = codecs.open('ocr_result/'+os.path.basename(str(row[1]['filename']))[:-4] + '.txt').readlines()[:]
    # 文字
    ocr_text = [x.split('\t')[1].strip() for x in ocrs]
    return ''.join(ocr_text).replace(' ', '')


In [13]:
%%time
train_text = [read_ocr(x) for x in train_df.iloc[:].iterrows()]

CPU times: user 6.7 s, sys: 238 ms, total: 6.94 s
Wall time: 6.93 s


In [14]:
train_df.loc[:, 'text'] = train_text

In [15]:
print(train_df.shape)
train_df = train_df[~train_df.apply(lambda x: len(x[-1]) == 0, axis=1)]
print(train_df.shape)

(41117, 6)
(41113, 6)


In [16]:
def str_index(doc, query):
    try:
        return doc.index(query), len(query)
    except:
        return 0, 0

In [17]:
train_df['index'] = train_df.apply(lambda x: str_index(x[-1], x[-2]), axis=1)

In [18]:
train_df['index_start'] = train_df['index'].apply(lambda x: x[0])
train_df['index_end'] = train_df['index'].apply(lambda x: x[1])

train_df['index_end'] = train_df['index_start'] + train_df['index_end']

In [19]:
train_df = train_df[train_df['index_end'] !=0]
train_df = train_df[train_df['index_end'] < 480]

In [21]:
from transformers import BertTokenizerFast, BertForQuestionAnswering

tokenizer = BertTokenizerFast.from_pretrained('hfl/chinese-bert-wwm-ext')
model = BertForQuestionAnswering.from_pretrained('hfl/chinese-bert-wwm-ext')

train_encodings = tokenizer(list(train_df['text'])[:], list(train_df['question_text'])[:], 
                            return_tensors='pt', truncation=True, padding=True,
                           max_length=512)

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl/chinese-

In [22]:
train_encodings['start_positions'] = [train_encodings.char_to_token(idx, x) for idx, x in enumerate(train_df['index_start'].values[:])]
train_encodings['end_positions'] = [train_encodings.char_to_token(idx, x-1) for idx, x in enumerate(train_df['index_end'].values[:])]

In [23]:
[x for x in train_encodings['end_positions'] if x == None]

[]

In [24]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
    
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)

In [25]:
from torch.utils.data import DataLoader
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for idx, batch in enumerate(train_loader):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()

        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)
        acc1 = ( (start_pred == start_positions).sum() / len(start_pred) ).item()
        acc2 = ( (end_pred == end_positions).sum() / len(start_pred) ).item()

        if idx % 10 == 0:
            print(loss.item(), acc1, acc2)
            with codecs.open('log.log', 'a') as up:
                up.write('{3}\t{0}\t{1}\t{2}\n'.format(loss.item(), acc1, acc2, 
                                                       str(epoch) + '/' + str(idx) +'/'+ str(len(train_loader))))

model.eval()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


6.392827987670898 0.0 0.0
4.641648769378662 0.0 0.0
3.6394097805023193 0.125 0.125
3.7527449131011963 0.125 0.25
4.01697301864624 0.0 0.0
5.268149375915527 0.125 0.125
3.655611753463745 0.25 0.25
3.401010036468506 0.125 0.125
2.8646187782287598 0.0 0.0
2.3008298873901367 0.375 0.625
3.1255717277526855 0.5 0.25
4.104433536529541 0.25 0.125
1.9003040790557861 0.375 0.75
3.4407365322113037 0.125 0.25
2.717897415161133 0.625 0.375
2.4693455696105957 0.625 0.5
2.2299017906188965 0.5 0.375
2.390782356262207 0.5 0.5
1.9221857786178589 0.625 0.375
2.609260320663452 0.375 0.375
1.9003806114196777 0.625 0.375
1.879948616027832 0.375 0.375
2.43220853805542 0.5 0.375
2.430473804473877 0.375 0.375
2.999849796295166 0.25 0.0
2.43666934967041 0.625 0.375
2.276320457458496 0.5 0.5
3.106001853942871 0.5 0.375
1.8396708965301514 0.375 0.25
2.252828359603882 0.5 0.5
2.288601875305176 0.375 0.375
2.990363597869873 0.25 0.375
1.7852578163146973 0.5 0.5
2.253142833709717 0.5 0.5
1.2141497135162354 0.5 0.25


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [17]:
torch.save(model, 'bert-qa.pt')

In [18]:
test_df = pd.read_csv('测试 第 1 轮/test1-utf8.csv')
test_df['filename'] = 'input/训练集/image/' + df['filename'] # 改为本地路径

test_text = [read_ocr(x) for x in test_df.iloc[:].iterrows()]

In [19]:
test_df['text'] = test_text

In [20]:
def predcit(doc, query):
    item = tokenizer([doc, query], max_length=512, return_tensors='pt', truncation=True, padding=True,)
    with torch.no_grad():
        input_ids = torch.tensor(item['input_ids']).to(device).reshape(1, -1)
        attention_mask = torch.tensor(item['attention_mask']).to(device).reshape(1, -1)
        
        outputs = model(input_ids[:, :512], attention_mask[:, :512])
        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)
    
    try:
        start_pred = item.token_to_chars(0, start_pred)
        end_pred = item.token_to_chars(0, end_pred)
    except:
        return ''
    
    if start_pred.start > end_pred.end:
        return ''
    else:
        return doc[start_pred.start:end_pred.end]

In [21]:
predcit(train_df['text'].iloc[25], train_df['question_text'].iloc[25])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


'4L'

In [23]:
train_df.iloc[25]

index                                                      (53, 2)
question_id                                                 Q00049
filename                      input/训练集/image/61fd813500f821f8.jpg
question_text                                            产品的容量有多大？
answer_text                                                     4L
text             产品参数产品名称：苏泊尔（SUPOR产品型号：SF40FC663额定电压：220V额定频率：...
index_start                                                     53
index_end                                                       55
Name: 48, dtype: object

In [24]:
import json
import codecs
from tqdm import tqdm

dw_label = []
for row in tqdm(test_df.iloc[:].iterrows()):
    dw_label.append({'questionId': row[1].question_id, 'answer': predcit(row[1].text, row[1].question_text)})
    
import json
with open('answer.json', 'w') as up:
    json.dump(dw_label, up)

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
8615it [03:12, 44.84it/s]


In [25]:
!zip -r answer.zip answer.json

updating: answer.json (deflated 90%)
